<a href="https://colab.research.google.com/github/Stanislav98V/Speech_synthesis/blob/main/1laba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки

In [1]:
!pip install lxml

In [2]:
!pip install bs4

In [3]:
from google.colab import drive
from html.parser import HTMLParser
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score

Подключаем диск

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


Загружаем файлы

In [5]:
with open('/content/drive/MyDrive/3_семестр/Синтез_речи/Text_Вобликов.xml', 'r') as f:
    line = f.read()
soup = BeautifulSoup(line, features="html.parser")

In [18]:
with open('/content/drive/MyDrive/3_семестр/Синтез_речи/Test!.xml', 'r') as f:
    line = f.read()
soup_test = BeautifulSoup(line, features="html.parser")

# Функции

Фукция для выделения признаков из слов

In [7]:
def fea_word(tag_word):
    feats = []
    if 'subpart_of_speech' in tag_word.dictitem.attrs:
        feats.append(tag_word.dictitem.attrs['subpart_of_speech'])
    else:
        feats.append(0)
    if 'form' in tag_word.dictitem.attrs:
        feats.append(tag_word.dictitem.attrs['form'])
    else:
        feats.append(0)
    if 'genesys' in tag_word.dictitem.attrs:
        feats.append(tag_word.dictitem.attrs['genesys'])
    else:
        feats.append(0)
    if 'stress_dict' in tag_word.dictitem.attrs:
        feats.append(tag_word.dictitem.attrs['stress_dict'])
    else:
        feats.append(0)
    if 'semantics1' in tag_word.dictitem.attrs:
        feats.append(tag_word.dictitem.attrs['semantics1'])
    else:
        feats.append(0)
    if 'semantics2' in tag_word.dictitem.attrs:
        feats.append(tag_word.dictitem.attrs['semantics2'])
    else:
        feats.append(0)
    return feats

Фукция для выделения признаков из пауз

In [8]:
def fea_pause(tag_word):
    feats = []
    feats.append(['pause'])
    feats.append(tag_word['time'])
    return feats

Фукция для выделения признаков из знаков препинания

In [9]:
def fea_content(tag_word):
    feats = []
    feats.append(['content'])
    if len(tag_word.attrs) > 0:
        if 'punktend' in tag_word.attrs:
            feats.append(tag_word.attrs['punktend'])
        else:
            feats.append(0)
        if 'punktbeg' in tag_word.attrs:
            feats.append(tag_word.attrs['punktbeg'])
        else:
            feats.append(0)
        if 'emphend' in tag_word.attrs:
            feats.append(tag_word.attrs['emphend'])
        else:
            feats.append(0)
        if 'emphbeg' in tag_word.attrs:
            feats.append(tag_word.attrs['emphbeg'])
        else:
            feats.append(0)
    else:
        for i in range(4):
            feats.append(0)
    return feats

Фукция для парсинга 

In [10]:
def parse(soup):
    sents = []
    tags_sents = soup.find_all(['sentence'])
    for tag_sent in tqdm(tags_sents):
        tags_words = tag_sent.find_all(['pause', 'word', 'content'])
        for i in range(len(tags_words)):
            if tags_words[i].name == 'word':
                if 'original' in tags_words[i].attrs:
                    wor = []
                    wor.append(['Слово= ', tags_words[i].attrs['original']])
                    if tags_words[i].dictitem != None:
                        feats_word = fea_word(tags_words[i])
                    else:
                        feats_word = []
                        for i in range(6):
                            feats_word.append(0)
                    if 'nucleus' in tags_words[i].attrs:
                        feats_word.append(tags_words[i].attrs['nucleus'])
                    else:
                        feats_word.append(0)
                    if tags_words[i].letter != None:
                        if 'flag' in tags_words[i].letter.attrs:
                            feats_word.append(tags_words[i].letter.attrs['flag'])
                        else:
                            feats_word.append(0)
                    else:
                        feats_word.append(0)
                    wor.append(feats_word)
                    if len(tags_words) > i + 1:
                        if tags_words[i + 1].name == 'pause':
                            feats_pause = fea_pause(tags_words[i + 1])
                            wor.append(feats_pause)
                        elif tags_words[i + 1].name == 'content':
                            feats_content = fea_content(tags_words[i + 1])
                            wor.append(feats_content)
                    if len(tags_words) > i + 2:
                        if tags_words[i + 2].name == 'pause':
                            feats_pause = fea_pause(tags_words[i + 2])
                            wor.append(feats_pause)
                        elif tags_words[i + 2].name == 'content':
                            feats_content = fea_content(tags_words[i + 2])
                            wor.append(feats_content)
                    if len(wor) == 3:
                        if wor[2][0] == ['content']:
                            fe = []
                            fe.append(['pause'])
                            fe.append(0)
                            wor.append(fe)
                        elif wor[2][0] == ['pause']:
                            copyz = wor[2]
                            del wor[2]
                            fe = []
                            fe.append(['content'])
                            for i in range(4):
                                fe.append(0)
                            wor.append(fe)
                            fe = []
                            fe.append(['pause'])
                            fe.append(copyz[1])
                            wor.append(fe)
                    elif len(wor) == 2:
                        fe = []
                        fe.append(['content'])
                        for i in range(4):
                            fe.append(0)
                        wor.append(fe)
                        fe = []
                        fe.append(['pause'])
                        fe.append(0)
                        wor.append(fe)
                    elif len(wor) == 4:
                        copyz = wor[2]
                        del wor[2]
                        fe = []
                        fe.append(['pause'])
                        fe.append(copyz[1])
                        wor.append(fe)
                    sents.append(wor)
    print('Файл распарсин')
    return sents

Функция для компенсации нулями недостающих признаков

In [11]:
def completion(sents):
    X = []
    paus = []
    dlin_paus = []
    udar = []
    for sent in tqdm(sents):
        line = []
        for i in range(8):
            if i == 6:
                if int(sent[1][i]) == 0:
                    udar.append(0)
                else:
                    udar.append(1)
            else:
                line.append(int(sent[1][i]))
        for i in range(1, 5):
            line.append(int(sent[2][i]))
        if int(sent[3][1]) == 0:
            dlin_paus.append(0)
        elif 300 > int(sent[3][1]) > 0:
            dlin_paus.append(1)
        else:
            dlin_paus.append(2)
        if sent[3][1] == 0:
            paus.append(0)
        else:
            paus.append(1)
        X.append(line)
    return X, paus, dlin_paus, udar

# Основа

Обработка тренировочного файла

In [12]:
sents = parse(soup)
print(len(sents))
print(sents[:10])
X, paus, dlin_paus, udar = completion(sents)

  0%|          | 0/8594 [00:00<?, ?it/s]

Файл распарсин
74858
[[['Слово= ', 'Многое'], ['1', '1', '6', '1', 0, '2', 0, '16'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'удивительное'], ['3', '33', '0', '3', 0, 0, 0, '8'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'в'], ['9', '0', '0', 0, 0, '100', 0, 0], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'жизни'], ['2', '12', '5', '1', 0, 0, 0, 0], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'начинается'], ['6', '63', '0', '3', 0, 0, 0, 0], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'с'], ['9', '0', '0', 0, 0, '100', 0, 0], [['content'], '4', 0, 0, 0], [['pause'], 0]], [['Слово= ', 'пустяков;'], ['2', '10', '4', '3', 0, 0, '2', 0], [['content'], 0, 0, 0, 0], [['pause'], '383']], [['Слово= ', 'В'], ['9', '0', '0', 0, 0, '100', 0, '16'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'данном'], ['3', '26', '0', '1', 0, 0, 0, 0], [['content'], '12', 0, 0, 0], [['pause'], 0]], [['Слово= ', 'случае -'], ['1',

  0%|          | 0/74858 [00:00<?, ?it/s]

In [13]:
print(len(dlin_paus), len(paus), len(udar), len(X), len(X[0]))

74858 74858 74858 74858 11


Обработка тестового файла

In [19]:
sents_test = parse(soup_test)
print(len(sents_test))
print(sents_test)
X_test, paus_test, dlin_paus_test, udar_test = completion(sents_test)

  0%|          | 0/200 [00:00<?, ?it/s]

Файл распарсин
3004
[[['Слово= ', 'Реально'], ['10', 0, 0, '2', 0, 0, 0, '16'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'начать'], ['6', '60', 0, '2', '100', '10', 0, '8'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'день'], ['1', '5', '4', '1', 0, 0, 0, '8'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'с'], ['9', 0, 0, 0, 0, '100', 0, 0], [['content'], '12', 0, 0, 0], [['pause'], 0]], [['Слово= ', 'улыбки -'], ['1', '2', '5', '2', 0, 0, 0, '8'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'это'], ['9', 0, 0, '1', 0, '400', 0, '8'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'проснуться'], ['6', '60', 0, '2', '100', 0, 0, '8'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'от'], ['9', 0, 0, 0, 0, '100', 0, '8'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'звонка'], ['1', '2', '4', '2', 0, 0, 0, '8'], [['content'], 0, 0, 0, 0], [['pause'], 0]], [['Слово= ', 'будильника'], ['1', '2', '4', '2

  0%|          | 0/3004 [00:00<?, ?it/s]

# Обучение

### Отдельное обучение трёх Random Forest

Random Forest для места пауз

In [15]:
from sklearn.ensemble import RandomForestClassifier

clf_pause = RandomForestClassifier(max_depth=10, random_state=4)
clf_pause.fit(X[:73000], paus[:73000])
predicted = clf_pause.predict(X[73000:])
print(metrics.classification_report(paus[73000:], predicted))

              precision    recall  f1-score   support

           0       0.87      0.88      0.88      1401
           1       0.62      0.60      0.61       457

    accuracy                           0.81      1858
   macro avg       0.75      0.74      0.74      1858
weighted avg       0.81      0.81      0.81      1858



Random Forest для длины пауз

In [16]:
clf_dlin = RandomForestClassifier(max_depth=10, random_state=4)
clf_dlin.fit(X[:73000], dlin_paus[:73000])
predicted = clf_dlin.predict(X[73000:])
print(metrics.classification_report(dlin_paus[73000:], predicted))

              precision    recall  f1-score   support

           0       0.78      0.98      0.87      1401
           1       0.86      0.04      0.07       161
           2       0.57      0.17      0.26       296

    accuracy                           0.77      1858
   macro avg       0.74      0.40      0.40      1858
weighted avg       0.75      0.77      0.71      1858



Random Forest для места ударений

In [17]:
clf_udar = RandomForestClassifier(max_depth=10, random_state=4)
clf_udar.fit(X[:73000], udar[:73000])
predicted = clf_udar.predict(X[73000:])
print(metrics.classification_report(udar[73000:], predicted))

              precision    recall  f1-score   support

           0       0.87      0.88      0.87      1389
           1       0.63      0.60      0.61       469

    accuracy                           0.81      1858
   macro avg       0.75      0.74      0.74      1858
weighted avg       0.81      0.81      0.81      1858



# JSON

In [20]:
udar_test = clf_udar.predict(X_test)
pause_test = clf_dlin.predict(X_test) + clf_pause.predict(X_test)

Функция для приведения результатов к нужному виду

In [21]:
def create_json(sents_test, X_test, udar_test, pause_test):
    s = []
    i = 0
    while i < len(sents_test):
        w = {'words': []}
        count = 0
        z = -1
        while z != 0 and i < len(sents_test):
            w_i = {"content": 0, "phrasal_stress": False, "pause_len": -1}
            w_i["content"] = sents_test[i][0][1]
            if udar_test[i] == 1:
                w_i["phrasal_stress"] = True
            if pause_test[i] == 1:
                w_i["pause_len"] = 200
            elif pause_test[i] >= 2:
                w_i["pause_len"] = 500
            w['words'].append(w_i)
            if count != 0:
                s.append(w)
                z = 0
            if X_test[i][7] == 1 or X_test[i][7] == 5 or X_test[i][7] == 6 or X_test[i][7] == 7:
                count = 1
            i += 1
    return s

Сохранение результатов в формате json

In [22]:
import json
json_file = create_json(sents_test, X_test, udar_test, pause_test)
to_json= json.dumps(json_file, ensure_ascii=False)

In [25]:
with open('/content/drive/MyDrive/3_семестр/Синтез_речи/test_file.json', 'w') as file:
    json.dump(to_json, file)

Результаты

In [31]:
to_json

'[{"words": [{"content": "Реально", "phrasal_stress": false, "pause_len": -1}, {"content": "начать", "phrasal_stress": false, "pause_len": -1}, {"content": "день", "phrasal_stress": true, "pause_len": 200}, {"content": "с", "phrasal_stress": false, "pause_len": -1}, {"content": "улыбки -", "phrasal_stress": true, "pause_len": 500}, {"content": "это", "phrasal_stress": false, "pause_len": -1}, {"content": "проснуться", "phrasal_stress": false, "pause_len": -1}, {"content": "от", "phrasal_stress": false, "pause_len": -1}, {"content": "звонка", "phrasal_stress": true, "pause_len": 500}, {"content": "будильника", "phrasal_stress": true, "pause_len": 500}, {"content": "в", "phrasal_stress": false, "pause_len": -1}, {"content": "одну", "phrasal_stress": false, "pause_len": -1}, {"content": "минуту", "phrasal_stress": true, "pause_len": 200}, {"content": "первого", "phrasal_stress": false, "pause_len": -1}, {"content": "ночи,", "phrasal_stress": true, "pause_len": 500}, {"content": "широко", 